In [1]:
import pandas as pd
import random
import networkx as nx
import matplotlib.pyplot as plt
import pickle
from pyvis.network import Network
import community.community_louvain as community_louvain
from tqdm import tqdm

---

In [2]:
# TODO : 
# Faire fonction de vérification des paires id: community pour le dataframe
# Revoir la méthode de sauvegarde des graphes (qui des anciens graphs)
# 

## Importation des données

In [3]:
project = pd.read_csv("../Data/CSV/Projects/project.csv", sep=";")

In [4]:
organization = pd.read_csv("../Data/CSV/Projects/organization.csv", sep=";")

On importera les pubications après

In [4]:
print(project.shape)
print(organization.shape)

(35381, 20)
(177190, 25)


---

## Création du graph

On récupère les ID unique des projets

In [48]:
project_id = list(project["id"].unique())

Pour chaque projet je récupère les id des organisations qui y participent. Tout ça dans un dictionnaire.

In [7]:
projet_orga = {}
for projet in project_id:
    projet_orga[projet] = list(organization[organization["projectID"] == projet]["organisationID"])

Création d'un graph vide

In [8]:
G = nx.Graph()

Fonction de création du graph (prendre plus d'une heure à faire tourner)

In [9]:
########### /!\ TOURNE INDEFINIEMENT /!\ ##############

yes_counter = 0
for i, tq in zip(range(1, len(projet_orga.keys())), tqdm(range(len(projet_orga.keys())))):
#     print(projet_orga[project_id[i]])
    for t in range(len(projet_orga.keys())):
#         G.add_node(project_id[i - 1])
        G.add_node(project_id[i - 1], label=project_id[i - 1])
        for y in projet_orga[project_id[i - 1]]:
#         print(f"i: {i} | i+1: {projet_orga[project_id[i+1]]} | y: {y}")
            if G.has_edge(project_id[i - 1], project_id[t]) is False:
                for x in projet_orga[project_id[t]]:
            
                    if y == x:
                        yes_counter += 1
                        G.add_edge(project_id[i-1], project_id[t])

G.remove_edges_from(nx.selfloop_edges(G))
# On supprime les boucles du graph car cela n'apporte pas d'information et cela prend de la place

  0%|                                                                              | 8/35381 [00:02<3:31:10,  2.79it/s]


KeyboardInterrupt: 

Pour importer un graph sauvegardé au format .pickle

In [5]:
 G = pickle.load(open("../graphs_save/graph_3_labeled.pickle", "rb"))

In [11]:
G.number_of_nodes()

35381

In [12]:
G.number_of_edges()

9982209

---

## Détection de communauté

### Détection de communauté

In [13]:
partition = community_louvain.best_partition(G)

In [19]:
print(f"Nombre de communautés : {len(set(partition.values()))}")

Nombre de communautés : 4377


On ajoute les communauté en tant qu'attributs aux noeuds du graph

In [34]:
nx.set_node_attributes(G, partition, 'partition')

### Sauvegarde des communautés

Pour sauvegarder le graph au format .pickle

In [35]:
# pickle.dump(G, open('../graphs_save/graphe_3_labeled.pickle', 'wb'))

Pour sauvegarder le graph en format gexf pour l'importer sur Gephi

In [ ]:
# nx.write_gexf(G, "../graphs_save/graphe_3_labeled.gexf")

On crée un DataFrame avec en index les id des noeuds et leur communauté

In [14]:
partition_df = pd.DataFrame.from_dict(partition, orient="index").astype({0:"int32"})

On ajoute les communautés au DataFrame de données

In [17]:
project = project.join(partition_df, on="id")
project.rename({0:"Community"}, axis=1, inplace=True)

In [18]:
project.head(2)

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,...,frameworkProgramme,masterCall,subCall,fundingScheme,nature,objective,contentUpdateDate,rcn,grantDoi,Community
0,817296,eDCaseMAN,CLOSED,"Defendable legal cases through an affordable, ...",2018-05-01,2018-09-30,71429,50000,H2020-EU.2.3.,EIC-SMEInst-2018-2020,...,H2020,H2020-EIC-SMEInst-2018-2020,H2020-SMEInst-2018-2020-1,SME-1,NaN,"""Electronic discovery (known as e-discovery) i...",2022-08-10 13:07:21,217713,10.3030/817296,0
1,764717,WinWind,CLOSED,Winning social acceptance for wind energy in w...,2017-10-01,2020-03-31,"2124462,5","2124462,5",H2020-EU.3.3.,LCE-21-2017,...,H2020,H2020-LCE-2016-2017,H2020-LCE-2017-RES-CSA,CSA,NaN,The overall objective of WinWind is to enhance...,2022-08-17 10:54:17,211548,10.3030/764717,1


### Modification et analyse des communautés

On calcule la proportion du chaque communauté par rapport à tous le data set 

In [32]:
proportions = project['Community'].value_counts(normalize=True).to_frame()
proportions.rename({"Community": "Proportion"}, inplace=True, axis=1)

In [33]:
proportions

,Proportion
1,0.250276
4,0.094599
16,0.073034
12,0.068596
7,0.063763
...,...
1639,0.000028
1640,0.000028
1641,0.000028
1642,0.000028


On recupère l'index (ici les communautés) pour lesquelles la proportion est inférieure à 1% du data set 

In [46]:
index_label = proportions[proportions["Proportion"] <= 0.01].index.to_list()

On modifie la valeur des communautés pour lesquelles la proportion est inférieure à 1%

In [62]:
def community_filter(x: int):
    if x in index_label:
        return -99
    else:
        return x

On test le code avec une copie du DataFrame (va changer à l'avenir)

In [63]:
project_prepared = project.copy()
project_prepared["Community"] = project["Community"].apply(community_filter)

In [72]:
project_prepared.head(3)

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,...,frameworkProgramme,masterCall,subCall,fundingScheme,nature,objective,contentUpdateDate,rcn,grantDoi,Community
0,817296,eDCaseMAN,CLOSED,"Defendable legal cases through an affordable, ...",2018-05-01,2018-09-30,71429,50000,H2020-EU.2.3.,EIC-SMEInst-2018-2020,...,H2020,H2020-EIC-SMEInst-2018-2020,H2020-SMEInst-2018-2020-1,SME-1,NaN,"""Electronic discovery (known as e-discovery) i...",2022-08-10 13:07:21,217713,10.3030/817296,-99
1,764717,WinWind,CLOSED,Winning social acceptance for wind energy in w...,2017-10-01,2020-03-31,"2124462,5","2124462,5",H2020-EU.3.3.,LCE-21-2017,...,H2020,H2020-LCE-2016-2017,H2020-LCE-2017-RES-CSA,CSA,NaN,The overall objective of WinWind is to enhance...,2022-08-17 10:54:17,211548,10.3030/764717,1
2,784994,PentaHelix,CLOSED,Multi stakeholder and governance approach for ...,2018-03-01,2021-09-30,"1808343,75","1808343,75",H2020-EU.3.3.,EE-09-2016-2017,...,H2020,H2020-EE-2016-2017,H2020-EE-2017-CSA-PPI,CSA,NaN,The PentaHelix project is focusing on developi...,2022-04-11 16:42:12,213566,10.3030/784994,1


In [78]:
project_prepared.shape

(35381, 21)

On crée un dictionnaire avec l'ID des projets et leurs nouvelles communautés pour être ajouté aux attributs des noeuds du graph

In [70]:
new_community = project_prepared.set_index('id')["Community"].to_dict()

In [83]:
# TODO: 
# Quid renommé les communautés par ordre (0, 1, 2 etc...) 
# Prendre les valeurs unique, les 'sorte' par ordre croissant et les modifié par
# leur index (garde le même ordre mais change les valeurs)

In [74]:
new_community

{817296: -99,
 764717: 1,
 784994: 1,
 841546: 1,
 716923: 15,
 814427: 1,
 751782: 3,
 727745: 1,
 740634: 4,
 771082: 1,
 894162: 1,
 678783: 3,
 101032680: 4,
 776740: 1,
 739676: -99,
 843269: 2,
 871647: -99,
 644821: 3,
 727524: 2,
 740829: 1,
 852169: 3,
 101029193: 16,
 882725: -99,
 759207: 16,
 890437: 1,
 101024588: 16,
 699298: 1,
 772705: 16,
 659083: 12,
 681463: -99,
 895260: 8,
 780473: 1,
 661479: 4,
 726904: -99,
 101024021: 3,
 883700: 7,
 794012: 2,
 874434: 1,
 756436: -99,
 101028525: 8,
 803335: 7,
 101007915: 3,
 640277: 1,
 845118: 1,
 876748: -99,
 836139: 7,
 638666: 2,
 802512: 1,
 808490: -99,
 799203: 16,
 704507: 2,
 748114: 7,
 847136: 1,
 810139: 16,
 101002921: 16,
 757247: 9,
 646453: 1,
 787068: 1,
 101026000: 12,
 649829: 1,
 642332: 4,
 704110: 1,
 651993: -99,
 728061: 16,
 842019: 1,
 727028: 1,
 891762: 2,
 832537: 1,
 682472: 16,
 714868: 4,
 753937: 7,
 792489: 3,
 952852: 1,
 957258: 1,
 725798: 9,
 847673: 1,
 661057: -99,
 747046: -99,
 713

In [81]:
print(f"Nombre de communautés après traitement : {len(set(new_community.values()))}")

Nombre de communautés après traitement : 16


On ajoute ces nouvelles communautés au graph ! 

In [75]:
nx.set_node_attributes(G, new_community, 'new_partition')

On enregistre ce nouveau graph

In [76]:
# pickle.dump(G, open('../graphs_save/graph_3_labeled_new_partition.pickle', 'wb'))

In [77]:
# nx.write_gexf(G, "../graphs_save/graph_3_labeled_new_partition.gexf")

---

## Importation des données de publications

In [7]:
publication_0 = pd.read_csv("../Data/CSV/Publications/projectPublications.csv", sep=";")
publication_1 = pd.read_csv("../Data/CSV/Publications/projectPublications_2.csv", sep=";")
publication_2 = pd.read_csv("../Data/CSV/Publications/projectPublications_3.csv", sep=";")
publication_3 = pd.read_csv("../Data/CSV/Publications/projectPublications_4.csv", sep=";")
publication_4 = pd.read_csv("../Data/CSV/Publications/projectPublications_5.csv", sep=";")
publication_5 = pd.read_csv("../Data/CSV/Publications/projectPublications_6.csv", sep=";")

In [9]:
print(publication_0.shape)
print(publication_1.shape)
print(publication_2.shape)
print(publication_3.shape)
print(publication_4.shape)
print(publication_5.shape)

(55001, 16)
(55001, 16)
(55001, 16)
(55001, 16)
(55001, 16)
(26586, 16)


In [14]:
publications_list = [ publication_0,
publication_1,
publication_2,
publication_3,
publication_4,
publication_5 ]

In [18]:
publication = pd.DataFrame()
for publi in publications_list:
    publication = pd.concat([publication, publi], axis=0)

In [20]:
publication.shape

(301591, 16)

In [22]:
publication.head()

,id,title,isPublishedAs,authors,journalTitle,journalNumber,publishedYear,publishedPages,issn,isbn,doi,projectID,projectAcronym,collection,contentUpdateDate,rcn
0,823965_1537977_PUBLI,On the Utility of High-Resolution Soil Moistur...,Peer reviewed articles,"Ait Hssaine, B.; Chehbouni, A.; Er-Raki, S.; K...",Remote Sensing,NaN,2021.0,NaN,2072-4292,NaN,10.3390/rs13040727,823965,ACCWA,Project publication,2022-09-27 18:18:00,872894
1,764958_1538020_PUBLI,Thermosensitive hydration of four acrylamide-b...,Peer reviewed articles,"Patrick K. Quoika, Maren Podewitz, Yin Wang, A...",The Journal of Physical Chemistry B,2,2020.0,9745 - 9756,1520-5207,NaN,10.1021/acs.jpcb.0c07232,764958,NANOSTEM,Project publication,2022-09-27 18:20:01,872958
2,677120_355435_PUBLI,On property (T) for Aut(F_n) and SL_n(Z),Peer reviewed articles,"Marek Kaluba, Dawid Kielak, Piotr W. Nowak",Annals of Mathematics,"Volume 193 (2021), Issue 2",2018.0,539-562,0003-486X,NaN,10.4007/annals.2021.193.2.3,677120,INDEX,Project publication,2022-09-30 20:15:15,544507
3,761042_1539423_PUBLI,Sporulation in solventogenic and acetogenic cl...,Peer reviewed articles,Mamou Diallo; Servé W. M. Kengen; Ana M. López...,Applied Microbiology and Biotechnology,105,2021.0,3533-3557,0175-7598,NaN,10.1007/s00253-021-11289-9,761042,BIOCONCO2,Project publication,2022-09-27 18:19:56,872951
4,871037_1533329_PUBLI,Comprehensive Fragment Screening of the SARS-C...,Peer reviewed articles,"Berg H, Wirtz Martin MA, Altincekic N, Alshaml...",Angew Chem Int Ed Engl.,NaN,2022.0,NaN,1433-7851,NaN,10.1002/anie.202205858,871037,iNEXT-Discovery,Project publication,2022-09-30 20:17:09,870820


On calcule le nombre de publications pour chaque projectID

In [32]:
print(len(list(publication["projectID"].value_counts().index)))
pub_project = publication["projectID"].value_counts().to_dict()

19395


In [27]:
pub_project

{633053: 6196,
 681202: 1593,
 696656: 942,
 785219: 876,
 785907: 607,
 881603: 554,
 665778: 512,
 669026: 461,
 654109: 441,
 945539: 428,
 654002: 414,
 654148: 414,
 739551: 409,
 739508: 379,
 692419: 379,
 654305: 378,
 690575: 376,
 643417: 361,
 641816: 351,
 689443: 327,
 789028: 307,
 668995: 305,
 870628: 304,
 761727: 304,
 727872: 295,
 830927: 293,
 654360: 289,
 823717: 279,
 727987: 275,
 663830: 269,
 720270: 267,
 633531: 266,
 654208: 263,
 681228: 251,
 730960: 249,
 654024: 248,
 643476: 243,
 678024: 240,
 692145: 239,
 641122: 238,
 730872: 231,
 664931: 231,
 688995: 230,
 824093: 223,
 633595: 220,
 739593: 218,
 692146: 218,
 691135: 216,
 739582: 215,
 694097: 214,
 713279: 213,
 871149: 211,
 691818: 210,
 713673: 210,
 653706: 207,
 777826: 207,
 692194: 206,
 734922: 205,
 690904: 203,
 691714: 202,
 654168: 201,
 692022: 200,
 712949: 200,
 666881: 199,
 668981: 199,
 733206: 199,
 952215: 198,
 678760: 195,
 713683: 193,
 810980: 192,
 741520: 191,
 817

In [34]:
id_project_publi = list(pub_project.keys())
print(len(id_project_publi))

19395


In [37]:
project[project["id"].isin(id_project_publi)]

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,ecSignatureDate,frameworkProgramme,masterCall,subCall,fundingScheme,nature,objective,contentUpdateDate,rcn,grantDoi
1,764717,WinWind,CLOSED,Winning social acceptance for wind energy in w...,2017-10-01,2020-03-31,"2124462,5","2124462,5",H2020-EU.3.3.,LCE-21-2017,2017-07-26,H2020,H2020-LCE-2016-2017,H2020-LCE-2017-RES-CSA,CSA,NaN,The overall objective of WinWind is to enhance...,2022-08-17 10:54:17,211548,10.3030/764717
3,841546,RES,TERMINATED,Rights for Ecosystem Services (RES): a framewo...,2020-01-13,2022-01-12,"224933,76","224933,76",H2020-EU.1.3.,MSCA-IF-2018,2019-04-11,H2020,H2020-MSCA-IF-2018,H2020-MSCA-IF-2018,MSCA-IF-EF-ST,NaN,Is currentIs current legal protection adequate...,2021-12-30 09:44:32,222283,10.3030/841546
4,716923,LEVIATHAN,CLOSED,Taming the Leviathan? Legal and Political Acco...,2017-08-01,2022-01-31,1184595,1184595,H2020-EU.1.1.,ERC-2016-STG,2016-11-29,H2020,ERC-2016-STG,ERC-2016-STG,ERC-STG,NaN,The Euro Crisis has had a transformative effec...,2022-09-08 18:14:15,206991,10.3030/716923
6,751782,GLEC-LAW,CLOSED,Global Ecological Custodianship: Innovative In...,2018-01-04,2020-01-03,"183454,8","183454,8",H2020-EU.1.3.,MSCA-IF-2016,2017-03-21,H2020,H2020-MSCA-IF-2016,H2020-MSCA-IF-2016,MSCA-IF-EF-ST,NaN,"International environmental law (IEL), with mu...",2022-08-17 10:35:28,209643,10.3030/751782
7,727745,Trillium II,CLOSED,Trillium Bridge II - Reinforcing the Bridges a...,2017-01-01,2019-06-30,"1104547,5","1000001,25",H2020-EU.3.1.,SC1-HCO-14-2016,2016-10-17,H2020,H2020-SC1-2016-2017,H2020-SC1-2016-CNECT,CSA,NaN,Trillium-II steps forward with an outstanding ...,2022-09-04 00:39:58,206099,10.3030/727745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35335,665860,SoMoPro 3,CLOSED,South Moravian Programme for Distinguished Res...,2015-09-01,2020-08-31,3658000,1829000,H2020-EU.1.3.,MSCA-COFUND-2014-FP,2015-06-25,H2020,H2020-MSCA-COFUND-2014,H2020-MSCA-COFUND-2014,MSCA-COFUND-FP,NaN,SoMoPro 3 is an innovated regional fellowship ...,2022-09-04 00:28:00,197906,10.3030/665860
35357,968850,SEP43,SIGNED,Innovative liquid chromatography columns for t...,2021-05-01,2023-04-30,"3415312,5","2390718,75",H2020-EU.2.3.,EIC-SMEInst-2018-2020,2021-05-27,H2020,H2020-EIC-SMEInst-2018-2020,H2020-EIC-SMEInst-2018-2020-3,SME-2b,NaN,SEPARATIVE SAS is an industrial startup locate...,2021-11-15 10:40:31,236497,10.3030/968850
35364,720579,TTMJ,CLOSED,NEW DIAPHRAGM WALL JOINT SYSTEM ALLOWING GREAT...,2016-09-01,2020-02-29,"2487218,4","1741052,88",H2020-EC,FTIPilot-1-2015,2016-07-29,H2020,H2020-FTIPilot-2015-1,H2020-FTIPilot-2015-1,IA,NaN,The increasing demand for complex infrastructu...,2022-08-16 23:12:08,204986,10.3030/720579
35372,777786,KANTINSA,SIGNED,Kant in South America,2018-01-01,2023-08-31,738000,544500,H2020-EU.1.3.,MSCA-RISE-2017,2017-11-14,H2020,H2020-MSCA-RISE-2017,H2020-MSCA-RISE-2017,MSCA-RISE,NaN,A consortium of 4 EU Universities and 4 Southe...,2022-09-04 01:04:33,212974,10.3030/777786


Nous n'avons pas tous les ID des projets car ne sont renseignés que les projets ayant publiés. 

In [61]:
def fill_publication(publi: dict, project_id: list):

    for project in project_id:
        if project in list(publi.keys()):
            continue
        else: 
            publi[project] = 0


In [62]:
fill_publication(pub_project, project_id)

La fonction ressortait un ID supplémentaire par rapport au nombre d'ID de projets qui nous avions. <br>
Voici un petit moyent de trouver quel ID est en trop et le supprimer

In [ ]:
count = 0
for intru in list(pub_project.keys()):
    if intru in project_id:
        continue
#         count += 1
    else: 
        to_delete = intru
        print(test)

In [71]:
del pub_project[743165]

Preuve qu'on ne l'a pas dans nos données : 

In [68]:
project[project["id"] == 743165]

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,ecSignatureDate,frameworkProgramme,masterCall,subCall,fundingScheme,nature,objective,contentUpdateDate,rcn,grantDoi
